# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
import json
import pandas as pd
import numpy as np 
import PIL

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sb

import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from collections import OrderedDict 
from torch.nn import functional as F

from torch.autograd import Variable


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# DONE: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.Resize(255),
                                       transforms.RandomCrop(224),
                                       transforms.RandomRotation(35),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])
                                      ])

val_test_transforms = transforms.Compose([transforms.Resize(255), 
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])
                                         ])

# DONE: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=val_test_transforms)
test_data = datasets.ImageFolder(test_dir, transform=val_test_transforms)

# DONE: Using the image datasets and the transforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=32)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
cat_to_name

{'21': 'fire lily',
 '3': 'canterbury bells',
 '45': 'bolero deep blue',
 '1': 'pink primrose',
 '34': 'mexican aster',
 '27': 'prince of wales feathers',
 '7': 'moon orchid',
 '16': 'globe-flower',
 '25': 'grape hyacinth',
 '26': 'corn poppy',
 '79': 'toad lily',
 '39': 'siam tulip',
 '24': 'red ginger',
 '67': 'spring crocus',
 '35': 'alpine sea holly',
 '32': 'garden phlox',
 '10': 'globe thistle',
 '6': 'tiger lily',
 '93': 'ball moss',
 '33': 'love in the mist',
 '9': 'monkshood',
 '102': 'blackberry lily',
 '14': 'spear thistle',
 '19': 'balloon flower',
 '100': 'blanket flower',
 '13': 'king protea',
 '49': 'oxeye daisy',
 '15': 'yellow iris',
 '61': 'cautleya spicata',
 '31': 'carnation',
 '64': 'silverbush',
 '68': 'bearded iris',
 '63': 'black-eyed susan',
 '69': 'windflower',
 '62': 'japanese anemone',
 '20': 'giant white arum lily',
 '38': 'great masterwort',
 '4': 'sweet pea',
 '86': 'tree mallow',
 '101': 'trumpet creeper',
 '42': 'daffodil',
 '22': 'pincushion flower',
 

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [5]:
# DONE: Build and train your network
model = models.resnet50(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/ec2-user/.torch/models/resnet50-19c8e357.pth
100.0%


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [6]:
#Iterates through the entire pre-trained network and turns off the gradient calculation and update, effectively freezing
#the network that has already been trained on ImageNet 
for param in model.parameters(): 
    param.requires_grad = False

#this adapts and tunes the head classifier 'fc' to fit this flower dataset with only 102 flower species, instead of 1000 classes
#this part of the model will be trained on the flower dataset
model.fc = nn.Sequential(OrderedDict([
                                    ('fc1', nn.Linear(2048, 512)),
                                    ('relu', nn.ReLU()),
                                    ('dropout', nn.Dropout(p=0.5)),
                                    ('fc2', nn.Linear(512, 102)),
                                    #the logarithm of the softmax probabilities is returned because of faster and 
                                    #usually more accurate computation. As computers use a base 2 number system, the computer
                                    #would store the float values of the softmax probabilities with imprecision
                                    #logarithm of the softmax improves numerical stability
                                    ('output', nn.LogSoftmax(dim=1))
                                    ]))

model.fc

Sequential(
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5)
  (fc2): Linear(in_features=512, out_features=102, bias=True)
  (output): LogSoftmax()
)

The validation function is largely taken from this udacity tutorial: https://classroom.udacity.com/nanodegrees/nd089/parts/4f56c240-e23f-49ae-9b84-b84c980fad9c/modules/47a93218-8271-457b-aca6-e2cdca10131b/lessons/e1eeafe1-2ba0-4f3d-97a0-82cbd844fdfc/concepts/43cb782f-2d8c-432e-94ef-cb8068f26042

In [7]:
# DONE: Do validation on the validation set
def validation(loader, device='cpu'):
    
    model.to(device) #sends to the network to either cpu ('cpu') or gpu ('cuda')
    model.eval() #turns dropout off to give an undistorted validation and test result
    
    #we use Negative Likelihood log loss function because the output from the model is log_Softmax
    criterion = nn.NLLLoss() 
    running_loss = 0 #initialization of loss score
    acc = 0 #initialization of accuracy 
    
    #the .no_grad ensures that the model does not calculate the gradient and update the weights during validation and testing
    with torch.no_grad():
        #the for loop to iterate over all images and labels in the validation and test sets 
        for images, labels in loader:
            #this sends the data input to either cpu ('cpu') or gpu ('cuda')
            images, labels = images.to(device), labels.to(device) 
            
            #forward pass and only forward pass
            output = model.forward(images)
            running_loss += criterion(output, labels).item()

            probs = torch.exp(output)
            
            #this is the comparison of the truth class index with the predicted class index to get the corrected prediciton
            #that is a list of 0s and 1s. 
            corrected_pred = (labels.data == probs.max(dim=1)[1])
            
            #here we get the accuracy by first converting the tensor a float tensor and apply the mean method
            #which sums up all the 1s and 0s, and divides by the total number of items
            acc += corrected_pred.type(torch.FloatTensor).mean()
    
    return running_loss/len(loader), acc/len(loader) * 100 

In [8]:
def train(epochs, optimizer, device='cpu'):   
    
    epochs = epochs 
    
    steps = 0 
    running_loss = 0
    acc = 0 
    
    model.to(device)#this gives the option of either cpu('cpu') or gpu('cuda')
    
    #Negative Log Likelihood Loss function, we use this because the output is in log form 
    criterion = nn.NLLLoss()

    for e in range(epochs):
        
        for idx, (images, labels) in enumerate(trainloader):
            
            steps += 1

            images, labels = images.to(device), labels.to(device)

            #sets the gradient to zero after each pass to prevent accumulating an inaccurately large gradient
            optimizer.zero_grad()

            output = model.forward(images)
            loss = criterion(output, labels)
            
            #Backward pass: where backward propagation occurs 
            loss.backward()
            
            #the weight update step where the optimizer applies the calculated negative gradient to weights and biases
            optimizer.step()
            
            #captures the loss during training, getting the scalar value from the tensor utilizing .item() method 
            running_loss += loss.item()
            
            #this converts the log ouput back to expected list of probability values between 0 and 1 using exponential function
            #the probs also contains a list of the corresponding class index second in the index at ps[1]
            probs = torch.exp(output)
            
            #this compares the correct class index (label.data) with the predicted class index (probs.max(dim=1)[1]) 
                                                                                    #associated with max probability
            corrected_pred = (labels.data == probs.max(dim=1)[1])
            
            #this captures the accuracy by taking the mean of all the comparisons that are either 1 for correct 
            #and 0 for not correct. The eq value is converted to a float tensor because it has the mean as a method 
            acc += corrected_pred.type(torch.FloatTensor).mean()

            if steps % 40 == 0:
                
                val_loss, val_acc = validation(validloader, 'cuda')
                
                print(                    #add 1 to e because range(epochs) starts at 0
                     'Epoch {}/{}'.format(e+1, epochs), 
                     'training loss: {:.4f}'.format(running_loss/40), 
                     'training accuracy: {:.4f}  '.format(acc/40 * 100),
                     #do not need to divide by length of validation set, already done in the function
                     'Validation loss: {:.4f}'.format(val_loss), 
                     'Validation accuracy: {}%'.format(val_acc)
                     )
                #sets the running_loss and accuracy back to zero so the loss and accuracy do not accumulate
                #and become too large, distorting the performance view during training
                running_loss = 0
                acc = 0 
            

In [9]:
#here was use the Adam optimizer with a learning rate of .001, 
#applies the Adam optimizer to only the head classifier of the pretrained network
optimizer = optim.Adam(model.fc.parameters(), lr = .001)

In [10]:
train(epochs=35, optimizer=optimizer, device='cuda')

Epoch 1/35 training loss: 4.5249 training accuracy: 6.3281   Validation loss: 4.2181 Validation accuracy: 11.658653259277344%
Epoch 1/35 training loss: 3.9424 training accuracy: 16.7969   Validation loss: 3.5939 Validation accuracy: 22.836538314819336%
Epoch 1/35 training loss: 3.2451 training accuracy: 32.5781   Validation loss: 2.7478 Validation accuracy: 39.0625%
Epoch 1/35 training loss: 2.4814 training accuracy: 44.2188   Validation loss: 2.2213 Validation accuracy: 46.501068115234375%
Epoch 1/35 training loss: 1.9162 training accuracy: 53.9844   Validation loss: 1.6644 Validation accuracy: 61.15117645263672%
Epoch 2/35 training loss: 1.4829 training accuracy: 64.7396   Validation loss: 1.4291 Validation accuracy: 61.61859130859375%
Epoch 2/35 training loss: 1.3169 training accuracy: 67.5781   Validation loss: 1.2752 Validation accuracy: 67.52136993408203%
Epoch 2/35 training loss: 1.1431 training accuracy: 72.1875   Validation loss: 1.2116 Validation accuracy: 69.8851547241211%
E

Epoch 14/35 training loss: 0.2312 training accuracy: 92.6302   Validation loss: 0.4698 Validation accuracy: 87.64690399169922%
Epoch 14/35 training loss: 0.2687 training accuracy: 91.7188   Validation loss: 0.4248 Validation accuracy: 88.87554168701172%
Epoch 14/35 training loss: 0.2192 training accuracy: 93.4375   Validation loss: 0.5016 Validation accuracy: 87.31304168701172%
Epoch 14/35 training loss: 0.2694 training accuracy: 92.1094   Validation loss: 0.4827 Validation accuracy: 87.76709747314453%
Epoch 14/35 training loss: 0.2817 training accuracy: 91.8750   Validation loss: 0.3917 Validation accuracy: 88.7553482055664%
Epoch 15/35 training loss: 0.2499 training accuracy: 92.6302   Validation loss: 0.4870 Validation accuracy: 87.04594421386719%
Epoch 15/35 training loss: 0.2644 training accuracy: 91.4844   Validation loss: 0.4210 Validation accuracy: 90.17094421386719%
Epoch 15/35 training loss: 0.2354 training accuracy: 93.1250   Validation loss: 0.4287 Validation accuracy: 88.9

Epoch 26/35 training loss: 0.2163 training accuracy: 93.2812   Validation loss: 0.4349 Validation accuracy: 88.78205108642578%
Epoch 26/35 training loss: 0.2498 training accuracy: 91.9531   Validation loss: 0.4783 Validation accuracy: 87.76709747314453%
Epoch 27/35 training loss: 0.1923 training accuracy: 93.8021   Validation loss: 0.3991 Validation accuracy: 88.84882354736328%
Epoch 27/35 training loss: 0.2119 training accuracy: 93.4375   Validation loss: 0.4777 Validation accuracy: 88.30128479003906%
Epoch 27/35 training loss: 0.1560 training accuracy: 94.6875   Validation loss: 0.4430 Validation accuracy: 89.62339782714844%
Epoch 27/35 training loss: 0.1790 training accuracy: 93.8281   Validation loss: 0.4050 Validation accuracy: 89.69017028808594%
Epoch 27/35 training loss: 0.2020 training accuracy: 93.7500   Validation loss: 0.5244 Validation accuracy: 86.80555725097656%
Epoch 28/35 training loss: 0.2124 training accuracy: 92.7083   Validation loss: 0.4330 Validation accuracy: 88.

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [11]:
#here we get the loss and accuracy on the test set to see if the model overfits, which it mostly doesn't
test_loss, test_acc = validation(testloader, 'cuda')

print('Testing loss:{:.4f}'.format(test_loss), 'Testing accuracy: {}%'.format(test_acc))

Testing loss:0.4451 Testing accuracy: 87.93016052246094%


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [12]:
#this dictionary captures the index associated with each class value; class values are keys, indices are values
model.class_to_idx = train_data.class_to_idx
model.class_to_idx

{'1': 0,
 '10': 1,
 '100': 2,
 '101': 3,
 '102': 4,
 '11': 5,
 '12': 6,
 '13': 7,
 '14': 8,
 '15': 9,
 '16': 10,
 '17': 11,
 '18': 12,
 '19': 13,
 '2': 14,
 '20': 15,
 '21': 16,
 '22': 17,
 '23': 18,
 '24': 19,
 '25': 20,
 '26': 21,
 '27': 22,
 '28': 23,
 '29': 24,
 '3': 25,
 '30': 26,
 '31': 27,
 '32': 28,
 '33': 29,
 '34': 30,
 '35': 31,
 '36': 32,
 '37': 33,
 '38': 34,
 '39': 35,
 '4': 36,
 '40': 37,
 '41': 38,
 '42': 39,
 '43': 40,
 '44': 41,
 '45': 42,
 '46': 43,
 '47': 44,
 '48': 45,
 '49': 46,
 '5': 47,
 '50': 48,
 '51': 49,
 '52': 50,
 '53': 51,
 '54': 52,
 '55': 53,
 '56': 54,
 '57': 55,
 '58': 56,
 '59': 57,
 '6': 58,
 '60': 59,
 '61': 60,
 '62': 61,
 '63': 62,
 '64': 63,
 '65': 64,
 '66': 65,
 '67': 66,
 '68': 67,
 '69': 68,
 '7': 69,
 '70': 70,
 '71': 71,
 '72': 72,
 '73': 73,
 '74': 74,
 '75': 75,
 '76': 76,
 '77': 77,
 '78': 78,
 '79': 79,
 '8': 80,
 '80': 81,
 '81': 82,
 '82': 83,
 '83': 84,
 '84': 85,
 '85': 86,
 '86': 87,
 '87': 88,
 '88': 89,
 '89': 90,
 '9': 91,
 '90

In [13]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [14]:
# DONE: Save the checkpoint 
#saves the number of epochs, the entire model, the optimizer, and the class_to_idx dictionary
checkpoint = {
              'epochs':35,
              'model':model, 
              'optimizer_state': optimizer.state_dict(),
               'class_to_idx': model.class_to_idx
             }

In [15]:
torch.save(checkpoint, 'checkpoint.pth') #saves the checkpoint as a pth file for inference later

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [16]:
# DONE: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(path):
    
    checkpoint = torch.load(path)#loads entire checkpoint as a dictionary
    model = checkpoint['model']#captures the model with weights and biases
    optimizer = checkpoint['optimizer_state']#captures the optimizer and its parameters 
    model.class_to_idx = checkpoint['class_to_idx']
    
    return checkpoint, optimizer, model.class_to_idx

checkpoint, optimizer, class_to_idx = load_checkpoint('checkpoint.pth')
checkpoint, optimizer

({'epochs': 35, 'model': ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Co

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

Source: https://stackoverflow.com/questions/16646183/crop-an-image-in-the-centre-using-pil for center cropping images to 224X224
I could have used the transforms module, but I thought it wise to also learn how to crop using PIL 

In [17]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    img = PIL.Image.open(image)
    
    min_leng = 256 #shortest side is 256 pixels, the minimum length
    ratio = min_leng/min(img.size) #this is used to retain the aspect ratio after cropping
    other_leng = int(max(img.size)*ratio) #this captures the longer side of the image
    img = img.resize((min_leng,other_leng), PIL.Image.ANTIALIAS) 
    
    #center cropping image to 224 X 224
    left = (min_leng - 224)/2
    top = (other_leng - 224)/2
    right = (min_leng + 224)/2
    bottom = (other_leng + 224)/2
    img = img.crop((left, top, right, bottom))

    #converts the PIL image to a numpy array and scales the image to range [0,1]
    np_img = np.array(img)/np.max(img)
    
    #means and standard deviations of the red, green, blue channels, respectively
    means = np.array([0.485, 0.456, 0.406])
    stds = np.array([0.229, 0.224, 0.225])
    
    #normalizes the image
    np_img = (np_img - means)/stds
    
    #transposes the image so the color channels are first for pytorch format
    np_img = np.ndarray.transpose(np_img)
    
    py_tensor = torch.from_numpy(np_img) #converts the numpy array into a pytorch tensor
    
    return py_tensor

#this just tests the output on a random image picked from google images
img = process_image('pink_flower.jpg')
img.shape

FileNotFoundError: [Errno 2] No such file or directory: 'pink_flower.jpg'

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image_path, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    image = process_image(image_path)
    image = image.numpy() #converts pytorch tensor to numpy array
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

imshow('flowers/test/101/image_07988.jpg');

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, device):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    img = process_image(image_path).float()
    img = img.unsqueeze_(0)
    
    checkpoint, _, class_to_idx = load_checkpoint('checkpoint.pth')
    
    model = checkpoint['model']
    
    model.eval() #turns dropout off 
    
    with torch.no_grad(): #gradient turned off because during prediction we want to know the result from training
        
        output = model.forward(img.to(device))
        probs, class_indices = F.softmax(output.data,dim=1).topk(5)
        
        probs = list(np.array(probs[0]))
        
        class_indices = list(np.array(class_indices[0]))
        
        classes = [k for k,v in checkpoint['class_to_idx'].items() if v in class_indices]
        
    return probs, classes

probs, classes = predict('flowers/test/101/image_07988.jpg', 'cuda')
print(probs)
print(classes)
    
    # DONE: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
plt.figure(figsize = [15,5])

#left plot
img_path = test_dir + '/101/image_07988.jpg'
ax1 = imshow(img_path, ax=plt.subplot(1, 2, 1))
ax1.set_title(cat_to_name['101'])

probs, classes = predict(img_path)
class_names = [cat_to_name[i] for i in classes]
#right plot
plt.subplot(1,2,2)
sb.barplot(x=probs, y = class_names, color='blue')
plt.xlabel('Probability')

plt.show();
    

